# Sentiment Analysis

Lots of libraries exist that will do sentiment analysis for you. Imagine that: just taking a sentence, throwing it into a library, and geting back a score! How convenient!

It's also **totally irresponsible** unless you know how the sentiment analyzer was built. In this homework we're going to see how sentiment analysis is done with a few packages.

## Installation

If you haven't already, you'll want to `pip install` two packages: NLTK and Textblob.

In [2]:
# !pip install nltk
#!pip install textblob

     |████████████████████████████████| 645kB 5.8MB/s eta 0:00:01


# NLTK: Natural Language Tooklit

[Natural Language Toolkit](https://www.nltk.org/) is the basis for a lot of text analysis done in Python. It's old and terrible and slow, but it's just been used for so long and does so many things that it's generally the default when people get into text analysis. The new kid on the block is [spaCy](https://spacy.io/) (but it doesn't do sentiment analysis so we're leaving it out of this).

When you first run NLTK, you need to download some datasets to make sure it will be able to do everything you want.

In [3]:
import nltk
nltk.download('vader_lexicon')
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/corina/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/corina/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /Users/corina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

To do sentiment analysis with NLTK, it only takes a couple lines of code:

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
sia.polarity_scores("This restaurant was great, but I'm not sure if I'll go there again.")

{'neg': 0.153, 'neu': 0.688, 'pos': 0.159, 'compound': 0.0276}

Asking `SentimentIntensityAnalyzer` for the `polarity_score` gave us four values in a dictionary:

- **negative:** the negative sentiment in a sentence
- **neutral:** the neutral sentiment in a sentence
- **positive:** the postivie sentiment in the sentence
- **compound:** the aggregated sentiment. 
    
Seems simple enough!

In [9]:
text = "I just got a call from my boss - does he realise it's Saturday?"
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Just like in real life, if you use an emoji you can be read as being more positive:

In [10]:
text = "I just got a call from my boss - does he realise it's Saturday? :)"
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.4588}

In [11]:
text = "I just got a call from my boss - does he realise it's Saturday? 😊"
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Why didn't it understand the emoji the same way it understood the emoticon? Well, **it only knows the words that it's been trained on,** and if VADER's never seen 😊 before it won't know what to think of it.

# TextBlob

TextBlob is built on top of NLTK, but is infinitely easier to use. It's still slow, but _it's so so so easy to use_. 

You can just feed TextBlob your sentence, then ask for a `.sentiment`!

In [12]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [13]:
blob = TextBlob("This restaurant was great, but I'm not sure if I'll go there again.")
blob.sentiment

Sentiment(polarity=0.275, subjectivity=0.8194444444444444)

**How could it possibly be easier than that?!?!?** This time we get a `polarity` and a `subjectivity` instead of all of those different scores, but it's basically the same idea.

If you like options: it turns out TextBlob actually has multiple sentiment analysis tools! How fun! We can plug in a different analyzer to get a different result.

In [14]:
blob = TextBlob("This restaurant was great, but I'm not sure if I'll go there again.", analyzer=NaiveBayesAnalyzer())
blob.sentiment

Sentiment(classification='pos', p_pos=0.5879425317005774, p_neg=0.41205746829942275)

Wow, that's a **very different result.** To understand why it's so different, we need to talk about where these sentiment numbers come from.

# But where do those numbers come from?

The most important thing to understand is **sentiment is always just an opinion.** In this case it's an opinion, yes, but specifically **the opinion of a machine.**

## VADER

NLTK's Sentiment Intensity Analyzer works is using something called **VADER**, which is a list of words that have a sentiment associated with each of them.

|Word|Sentiment rating|
|---|---|
|tragedy|-3.4|
|rejoiced|2.0|
|disaster|-3.1|
|great|3.1|

If you have more positives, the sentence is more positive. If you have more negatives, it's more negative. It can also take into account things like capitalization - you can read more about the classifier [here](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html), or the actual paper it came out of [here](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

**How do they know what's positive/negative?** They came up with a very big list of words, then asked people on the internet and paid them one cent for each word they scored.

## TextBlob's `.sentiment`

TextBlob's sentiment analysis is based on a separate library called [pattern](https://www.clips.uantwerpen.be/pattern).

> The sentiment analysis lexicon bundled in Pattern focuses on adjectives. It contains adjectives that occur frequently in customer reviews, hand-tagged with values for polarity and subjectivity.

Same kind of thing as NLTK's VADER, but it specifically looks at words from customer reviews.

**How do they know what's positive/negative?** They look at (mostly) adjectives that occur in customer reviews and hand-tag them.

## TextBlob's `.sentiment` + NaiveBayesAnalyzer

TextBlob's other option uses a `NaiveBayesAnalyzer`, which is a machine learning technique. When you use this option with TextBlob, the sentiment is coming from "an NLTK classifier trained on a movie reviews corpus."

**How do they know what's positive/negative?** Looked at movie reviews and scores using machine learning, see what words are associated with a positive/negative rating.

## What's this mean for me?

When you're doing automatic sentiment analysis, you have two major questions: 

* Where does the list of known words come from
* Where do the positive/negative scores come from

Let's compare the tools we've used so far.

|technique|word source|word selection|scores|
|---|---|---|---|
|NLTK (VADER)|everywhere|hand-picked|internet people, word-by-word|
|TextBlob|product reviews|hand-picked, mostly adjectives|internet people, word-by-word|
|TextBlob + NaiveBayesAnalyzer|movie reviews|all words|automatic based on score|

A major thing that should jump out at you is **how different the sources are.**

While VADER focuses on content found everywhere, TextBlob's two options are specific to certain domains. The [original paper for VADER](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf) passive-aggressively noted that VADER is effective at general use, but being trained on a specific domain can have benefits: 

> While some algorithms performed decently on test data from the specific domain for which it was expressly trained, they do not significantly outstrip the simple model we use.

They're basically saying, "if you train a model on words from a certain field, it will be good at that field."

## Questions

### Question 1: Is it okay to use a sentiment analyzer built on product reviews to check the sentiment of tweets? How about to check the sentiment of wine reviews?

In [ ]:
#People on twitter can be as angry and aggressive as dissapointed consumers, but I guess it would be better to avoid
#using it for tweets. Using it for wine reviews sounds proper.

### Question 2: Is it okay to use a sentiment analyzer trained on everything to check the sentiment of tweets? How about to check the sentiment of wine reviews?

In [ ]:
#It seems that it would work fine for both 

### Question 3: If I'm trying to report on whether people generally like or dislike what is happening throughout the Democratic debates, could I use these sorts of tools on tweets? Let's hear arguments for both sides.

In [ ]:
#I don't know if I would risk using these techniques for an actual story. The list of words may be thorough and big, 
#but the machine can't quite understand context. So there are chances that our analysis would be ambiguous. 

# Training our own sentiment analyzer

We don't want to rely on other people, we want to do this ourselves! There are two major ways to do sentiment analysis:

* Have a list of words that you humans assign positive or negative scores to
* Look at something scored (movie reviews, product reviews) and figure out which words appear with which scores

Depending on how you look at it, it's either a classification or a regression problem. We'll see the difference down below.

## Training on tweets

Let's say we were going to analyze the sentiment of tweets. If we had a list of tweets that were scored positive vs. negative, we could see which words are usually associated with positive scores and which are usually associated with negative scores.

Luckily, we have **Sentiment140** - http://help.sentiment140.com/for-students - a list of 1.6 million tweets along with a score as to whether they're negative (0) or positive (4). We'll use it to build our own machine learning algorithm to see separate positivity from negativity.

### Read in our data

In [56]:
import pandas as pd

columns = ['polarity', 'id', 'datetime', 'query', 'username', 'content']
df = pd.read_csv("trainingandtestdata/training.1600000.processed.noemoticon.csv", 
                 names=columns,
                 encoding='latin-1')
df = df.dropna()
df.head()

,polarity,id,datetime,query,username,content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Cleaning our data

The `polarity` field is whether something is positive or negative. How many do we have of each?.

In [57]:
df.polarity.value_counts()

4    800000
0    800000
Name: polarity, dtype: int64

In [58]:
df.polarity.dtypes

dtype('int64')

According to the documentation, `0` is negative and `4` is positive. Weird, right? Let's make it zero and one instead.

In [59]:
df.polarity = df.polarity.replace(4, 1)

Confirm you have 800k of each.

In [60]:
df.polarity.value_counts()

1    800000
0    800000
Name: polarity, dtype: int64

That is a **lot of tweets.**

Let's be honest: it's going to take our algorithms a long long time to process that many. Instead of working with our entire dataframe, let's use a **sample of 20,000**. If things are still slow before we can decrease this number.

* **Tip:** `df.sample(5)` will give you a sample of 5 elements of your dataframe

In [61]:
df = df.sample(20000)

Confirm you have 20,000 rows and 6 columns.

In [62]:
df.shape

(20000, 6)

## Vectorize our tweets

Create a `TfidfVectorizer` and use it to vectorize our tweets. Since we don't have all the time in the world, we should probably use `max_features` to only take a selection of terms - how about 2000 for now?

* **Tip:** Your end result should be a `words_df`

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
vectorizer = TfidfVectorizer(max_features = 2000)
vectors = vectorizer.fit_transform(df.content)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,00,000,09,10,100,1000,11,12,13,14,...,yours,yourself,youtube,yrs,yum,yummy,yup,zombie,zoo,à¹
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Your dataframe should look something like

|00|000|10|...|your|...|yummy|yup|½t|
|---|---|---|---|---|---|---|---|---|
|0.0|0.0|0.0|...|0.0|...|0.0|0.0|0.0|
|0.0|0.0|0.0|...|0.0|...|0.0|0.0|0.0|
|0.0|0.0|0.0|...|0.235754|...|0.0|0.0|0.0|
|0.0|0.0|0.0|...|0.0|...|0.0|0.0|0.0|


## Training our algorithm

### Setting up our variables

Create an `X` and a `y`, same as ever. In this case, what are our **features** and what are our **labels?**

In [77]:
X = words_df
y = df.polarity

Confirm that `X` has 20,000 rows and 2,000 columns, and that `y` has 20,000 rows of 1 column.

In [78]:
X.shape

(20000, 2000)

In [79]:
y.shape

(20000,)

### Picking an algorithm

What kind of algorithm do we want? We've used quite a few, and I just pulled another one couple classifiers of thin air.

In [118]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
%matplotlib inline

When picking an algorithm, think about what the output should be: is it a category? A probability, an amount? In this case **it might be any of those!**

For example:

* **A category:** `0` or `1` for negative vs positive
* **A probability:** The % chance that it's either negative or positive (between 0 and 1)
* **An amount:** A score between 0 and 1 about how positive it is

So hey, let's just make **one of each** of these. Name them `linreg`, `logreg`, `forest`, `svc`, and `bayes`.

The two new ones - `LinearSVC` and `MultinomialNB` - work exactly the same as your other classifiers, you'll be doing the standard creation and fitting:

```python
svc = LinearSVC()
svc.fit(X, y)
```

**Create and train classifiers in the cells below.** Add `%%time` to the top of each cell to see how long they take to train.

* **Tip:** Remember you need to add `C=1e9` to logistic regression, and specify the solver!
* **Tip:** If the logistic regression doesn't converge, it hasn't found an answer. You might need to increase `max_iter` (the default is 100)

In [82]:
from sklearn.model_selection import train_test_split

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [121]:
%%time
# Create and train a linear regression
svc = LinearSVC()
linreg = svc.fit(X_train, y_train)

CPU times: user 134 ms, sys: 4.41 ms, total: 138 ms
Wall time: 166 ms


In [122]:
linreg.score(X, y)

0.8019

In [123]:
prediction = svc.predict(X_test)
svc.get_params()


{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [157]:
%%time
# Create and train a logistic regression - if it doesn't converge be sure to increase max_iter
logreg = LogisticRegression(C=1e9, solver='lbfgs', max_iter=1000)
logreg.fit(X_train.head(7000), y_train.head(7000))

CPU times: user 23 s, sys: 413 ms, total: 23.4 s
Wall time: 13.4 s


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [127]:
import numpy as np

feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
65,aha,127.843972,3.326072e+55
123,apple,106.848263,2.532860e+46
448,dat,103.630553,1.014326e+45
1517,shop,92.178337,1.077812e+40
1466,saved,90.582213,2.184517e+39
875,hug,87.185716,7.316030e+37
1968,xd,75.938195,9.539795e+32
278,building,73.850140,1.182247e+32
1197,nah,72.032584,1.920232e+31
666,flying,70.932307,6.390125e+30


In [159]:
%%time
# Create and train a random forest classifier
random_for = RandomForestClassifier(n_estimators=10)
random_for.fit(X_train, y_train)

CPU times: user 4.04 s, sys: 101 ms, total: 4.14 s
Wall time: 4.45 s


In [129]:
feature_names = X.columns
importances = clf.feature_importances_

pd.DataFrame({
    'feature': feature_names,
    'feature importance': importances,
}).sort_values(by='feature importance', ascending=False)

,feature,feature importance
1987,you,1.799613e-02
1235,not,1.450370e-02
1224,no,1.156147e-02
1192,my,1.088657e-02
1144,miss,1.039346e-02
741,good,1.027232e-02
1454,sad,1.007804e-02
1743,to,1.000452e-02
1703,the,9.819631e-03
1699,thanks,9.501061e-03


In [131]:
%%time
# Create and train a linear support vector classifier (LinearSVC)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:
%%time
# Create and train a multinomial naive bayes classifier (MultinomialNB)


**How long did each take to train?** How much faster were some compared to others?

# Use our models on some new data

Now that we've trained our models, **they can try to predict whether a model is positive or negative**.

**Add five more sentences to the list below.** They should be a mix of positive and negative. They can be boring, they can be exciting, they can be short, they can be long.

In [132]:
# Create some test data

pd.set_option("display.max_colwidth", 200)

unknown = pd.DataFrame([
       "I'm not sure how I feel about toast",
       "Did you see the baseball game yesterday?",
       "The package was delivered late and the contents were broken",
       "Trashy television shows are some of my favorites",
       "I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",
       "I find chirping birds irritating, but I know I'm not the only one"
], columns=['content'])
unknown

,content
0,I'm not sure how I feel about toast
1,Did you see the baseball game yesterday?
2,The package was delivered late and the contents were broken
3,Trashy television shows are some of my favorites
4,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it."
5,"I find chirping birds irritating, but I know I'm not the only one"


First we need to **vectorizer** our sentences into numbers, so the algorithm can understand them.

Our algorithm only knows **certain words.** Run `vectorizer.get_feature_names()` to show you the list of the words it knows.

In [144]:
vectorizer.get_feature_names()

['00',
 '000',
 '09',
 '10',
 '100',
 '1000',
 '11',
 '12',
 '13',
 '14',
 '140',
 '15',
 '16',
 '17',
 '18',
 '1st',
 '20',
 '2009',
 '22',
 '23',
 '24',
 '25',
 '2am',
 '2day',
 '2morrow',
 '2nd',
 '2nite',
 '30',
 '33',
 '3am',
 '3gs',
 '3rd',
 '40',
 '45',
 '4th',
 '50',
 '6am',
 'able',
 'about',
 'absolutely',
 'abt',
 'accident',
 'according',
 'account',
 'ache',
 'across',
 'acting',
 'actually',
 'adam',
 'add',
 'added',
 'addicted',
 'adorable',
 'advice',
 'afford',
 'afraid',
 'after',
 'afternoon',
 'again',
 'against',
 'age',
 'ages',
 'ago',
 'agree',
 'ah',
 'aha',
 'ahaha',
 'ahead',
 'ahh',
 'ahhh',
 'ahhhh',
 'ahhhhh',
 'aim',
 'ain',
 'aint',
 'air',
 'airport',
 'al',
 'alas',
 'album',
 'alex',
 'alive',
 'all',
 'allowed',
 'almost',
 'alone',
 'along',
 'alot',
 'already',
 'alright',
 'also',
 'although',
 'always',
 'am',
 'amazing',
 'amazon',
 'america',
 'american',
 'amp',
 'amy',
 'an',
 'and',
 'andy',
 'angel',
 'angry',
 'anniversary',
 'annoying',


Usually when we use the vectorizer, we write code like this:
    
```python
vectors = vectorizer.fit_transform(....)
```

Which both learns all the words **and** counts them. In this case **we already have the list of words we know, we only want to count them.** So instead of `.fit_transform`, we just use `.transform`:

```python
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = ......
```

Finish making your `unknown_words_df` in the cell below.

In [146]:
unknown_words_df = vectorizer.transform(unknown.content)
vectors

<6x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 40 stored elements in Compressed Sparse Row format>

Confirm `unknown_words_df` is 11 rows and 2,000 columns.

In [147]:
unknown_words_df.shape

(6, 2000)

### Predicting with our models

To make a prediction for each of our sentences, you can use `.predict` with each of our models. For example, it would look like this for linear regression:

```python
unknown['pred_linreg'] = linreg.predict(unknown_words_df)
```

To add the prediction for logistic regression, you'd run similar `.predict` code, which will give you a `0` (negative) or a `1` (positive). A difference between the two is that for logistic regression, you can **also ask for the probability that the sentence is in the `1` category** instead of just simply the category. To do that, you use this code:

```python
unknown['pred_logreg_prob'] = linreg.predict_proba(unknown_words_df)[:,1]
```

**Add new columns for each of the models you trained.** If the model has a `.predict_proba`, add that as a column as well. 

* **Tip:** Tab is helpful for knowing whether `.predict_proba` is an option.
* **Tip:** Don't forget the `[:,1]` after `.predict_proba`, it means "give me the probability for category `1`

In [148]:
unknown['pred_linreg'] = linreg.predict(unknown_words_df)

In [158]:
unknown['pred_logreg'] = logreg.predict(unknown_words_df)

In [160]:
unknown['pred_random'] = random_for.predict(unknown_words_df)

,content,pred_linreg,pred_logreg,pred_random
0,I'm not sure how I feel about toast,0,0,0
1,Did you see the baseball game yesterday?,1,1,1
2,The package was delivered late and the contents were broken,0,0,0
3,Trashy television shows are some of my favorites,0,1,0
4,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",1,1,0
5,"I find chirping birds irritating, but I know I'm not the only one",0,0,0


Your output should look something like the below. Check your column names to confirm they match up.

|content|pred_linreg|pred_logreg|pred_logreg_proba|pred_forest|pred_forest_proba|pred_svc|pred_bayes|pred_bayes_proba|
|---|---|---|---|---|---|---|---|---|
|I'm not sure how I feel about toast|0.342560|0|0.271403|0|0.5|0|0|0.425271|
|...|...|...|...|...|...|...|...|...|

# Questions

### Question 4: What do the numbers mean? What's the difference between a 0 and a 1? A 0.5? Negative numbers?

### Question 5: Were there any sentences where the classifiers seemed to disagree about? How do you feel about the amount they disagree? 

### Question 6: What's the difference between using a 0/1 to talk about sentiment compared to 0-1? When might you use one compared to another?


### Question 7: What's the difference between the linear regression model and the other models we're using? Why might it fit or not fit?

### Question 8: Between 0-1, what range do you think counts as "negative," "positive" and "neutral"?

### Question 9: Does the variation in scores reflect the variation you would see among people? Or is it better or worse?

# Maybe we should have tested this?

We can actually see **which model performs the best**. Let's remind ourselves what we have by looking at:

* `X`
* `y`
* `df`

Our original dataframe is a list of many, many tweets. We turned this into `X` - vectorized words - and `y` - whether the tweet is negative or positive.

Before we used `.fit(X, y)` to train on all of our data. Instead, **we can test our models** by doing a test/train split and see if the predictions match the actual labels.

## Create test and training data 

Split your `X` and `y` into train and test datasets. I always have to look up how to do it, so here's the code for you:

```python
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
```

Use `X_train` and `y_train` to train all your models, except the linear regression one. You should be training:

* `logreg`
* `forest`
* `svc`
* `bayes`

Again, do them each in **separate cells** and use `%%time` to see how long each one takes to learn what's a positive vs negative tweet.

# Confusion matrices

To see how well they did, we'll use a confusion matrix for each one. For example, here is what you'll use for logistic regression:

```python
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)
```

In [ ]:
from sklearn.metrics import confusion_matrix

### Logistic Regression

### Random forest

### SVC

### Multinomial Naive Bayes

## Percentage-based confusion matrices

Those are kind of irritating in that they're just numbers. It might work better if you do something like this instead to get percentages:

```python
y_true = y_test
y_pred = logreg.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names) / matrix.sum(axis=1)
```

### Logisitic

### Random forest

### SVC

### Multinomial Naive Bayes

### Question 10: Which models performed the best? Were there big differences?

### Question 11: Do you think it's more important to be sensitive to negativity or positivity? Do we want more positive things incorrectly marked as negative, or more negative things marked as positive?

### Question 12: They all had very different training times. Which ones offer the best combination of performance and not making you wait around for an hour?

### Question 13: If you have a decent algorithm that trains more quickly, that could that mean about feature selection or the size of your training set? Why did we use `max_features=` and `df.sample`?

### Question 14: How do you feel about sentiment analysis?

### Question 15: How do you feel about [this piece from the UpShot](https://www.nytimes.com/interactive/2017/02/28/upshot/trump-sounds-different-tone-in-first-address-to-congress.html) that uses [the Emotional Lexicon](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm)?

### Question 16: What would you feel comfortable using our sentiment classifier for?